In [ ]:
import pandas as pd
import numpy as np
import hopsworks
from datetime import datetime

# Read and clean the data
credit_cards_df = pd.read_csv("Deviation_from_Policy_dataset.csv")

# Create a copy for cleaning
cleaned_df = credit_cards_df.copy()

# Add a transaction_id as primary key
cleaned_df['transaction_id'] = np.arange(len(cleaned_df))

# Replace negative distances with absolute values
cleaned_df['distance_from_home'] = cleaned_df['distance_from_home'].abs()
cleaned_df['distance_from_last_transaction'] = cleaned_df['distance_from_last_transaction'].abs()

# Add timestamp column (since we're dealing with a static dataset, we'll use current time)
cleaned_df['timestamp'] = datetime.now()

